# CAR brandclassification

### Transfer Learning VGG 16 and VGG 19 using Keras

### car brand classification Train data and test data
Train data consists of - 3 brand cars -audi,lamborghini, mercedes class
each folder consists of audi - 20 images; lamborghini - 19 cars; mercedes - 25

Test data consists of -audi - 9 ; lamborghini - 30 ; mercedes - 19

### what is transfer learning 
Transfer learning is a research problem in machine learning that focuses on storing knowledge 
gained while solving one problem and applying it to a different but related problem.

Transfer learning is a machine learning method where a model developed for a task is reused 
as the starting point for a model on a second task.

### How to Use Transfer Learning?
You can use transfer learning on your own predictive modeling problems.

Two common approaches are as follows:

### Develop Model Approach
### Pre-trained Model Approach
### Develop Model Approach
Select Source Task. You must select a related predictive modeling problem with an abundance of data where there is some relationship in the input data, output data, and/or concepts learned during the mapping from input to output data.
Develop Source Model. Next, you must develop a skillful model for this first task. The model must be better than a naive model to ensure that some feature learning has been performed.
Reuse Model. The model fit on the source task can then be used as the starting point for a model on the second task of interest. This may involve using all or parts of the model, depending on the modeling technique used.
Tune Model. Optionally, the model may need to be adapted or refined on the input-output pair data available for the task of interest.
### Pre-trained Model Approach
Select Source Model. A pre-trained source model is chosen from available models. Many research institutions release models on large and challenging datasets that may be included in the pool of candidate models from which to choose from.
Reuse Model. The model pre-trained model can then be used as the starting point for a model on the second task of interest. This may involve using all or parts of the model, depending on the modeling technique used.
Tune Model. Optionally, the model may need to be adapted or refined on the input-output pair data available for the task of interest.
This second type of transfer learning is common in the field of deep learning.

In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
# IMAGE_SIZE = [224, 224] - width and height

IMAGE_SIZE = [224, 224]
train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
# IMAGE_SIZE + [3] - 3 channels RGB
# include_top = False we have to include our own dataset as input and output our own number of classes  

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 493s 5us/step


In [ ]:
resnet.summary

In [ ]:
# don't train existing weights - reuse those weights from resnet
# don't retrain the mdoel just reuse all layers but not last layer
#
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
  # useful for getting number of output classes
folders = glob('Datasets/train/*')

In [ ]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [16]:
# view the structure of the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [17]:
#compile the model
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [18]:
# Use the Image Data Generator to import the images from the dataset
# in test data we should not use augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 64 images belonging to 3 classes.


In [20]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [21]:
# fit the model
# Run the cell. It will take some time to execute
# what is epoch - 
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50
2/2 [==============================] - 25s 13s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 26.6794 - val_acc: 0.3276
Epoch 2/50
2/2 [==============================] - 24s 12s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 27.1257 - val_acc: 0.3276
Epoch 3/50
2/2 [==============================] - 23s 12s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 27.2658 - val_acc: 0.3276
Epoch 4/50
2/2 [==============================] - 23s 12s/step - loss: 1.8681e-06 - acc: 1.0000 - val_loss: 26.8975 - val_acc: 0.3276
Epoch 5/50
2/2 [==============================] - 23s 12s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 27.8014 - val_acc: 0.3276
Epoch 6/50
2/2 [==============================] - 24s 12s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 28.3173 - val_acc: 0.3276
Epoch 7/50
2/2 [==============================] - 25s 13s/step - loss: 3.7253e-09 - acc: 1.0000 - val_loss: 28.2137 - val_acc: 0.3276
Epoch 8/50
2/2 [==============================] - 23s 12s/step

Epoch 28/50
2/2 [==============================] - 29s 15s/step - loss: 1.6764e-08 - acc: 1.0000 - val_loss: 38.2052 - val_acc: 0.3276
Epoch 29/50
2/2 [==============================] - 29s 15s/step - loss: 1.7624e-04 - acc: 1.0000 - val_loss: 37.7374 - val_acc: 0.3276
Epoch 30/50
2/2 [==============================] - 29s 14s/step - loss: 0.0013 - acc: 1.0000 - val_loss: 37.4502 - val_acc: 0.3276
Epoch 31/50
2/2 [==============================] - 25s 13s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 37.6763 - val_acc: 0.3276
Epoch 32/50
2/2 [==============================] - 24s 12s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 37.7078 - val_acc: 0.3276
Epoch 33/50
2/2 [==============================] - 24s 12s/step - loss: 1.8626e-09 - acc: 1.0000 - val_loss: 37.0114 - val_acc: 0.3276
Epoch 34/50
2/2 [==============================] - 24s 12s/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 36.8157 - val_acc: 0.3276
Epoch 35/50
2/2 [==============================] - 24s 12s/

In [ ]:
# can see loss,validation loss,valicdation accuracy, accuracy...
r.history

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [23]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')

In [24]:
# predict on test data
y_pred = model.predict(test_set)


In [25]:
# probability based on 3 - classes 
y_pred

array([[6.2281111e-26, 1.2584345e-25, 1.0000000e+00],
       [5.7013246e-25, 5.0343003e-25, 1.0000000e+00],
       [4.4075246e-25, 8.6745995e-25, 1.0000000e+00],
       [3.8885883e-25, 2.1096852e-25, 1.0000000e+00],
       [7.6663789e-26, 1.7028080e-25, 1.0000000e+00],
       [4.7028594e-25, 7.3667002e-25, 1.0000000e+00],
       [5.7185507e-26, 1.9625985e-25, 1.0000000e+00],
       [1.8259554e-24, 2.9903816e-24, 1.0000000e+00],
       [1.0704353e-23, 5.8202998e-24, 1.0000000e+00],
       [2.1425821e-24, 2.4288455e-24, 1.0000000e+00],
       [2.4663684e-25, 3.6616004e-24, 1.0000000e+00],
       [3.2171927e-25, 1.4625018e-25, 1.0000000e+00],
       [7.2013155e-27, 1.2389013e-24, 1.0000000e+00],
       [3.2196129e-24, 2.7823067e-24, 1.0000000e+00],
       [4.1639105e-26, 1.1540250e-25, 1.0000000e+00],
       [4.1148344e-26, 1.0516019e-25, 1.0000000e+00],
       [1.8475206e-25, 8.4782068e-26, 1.0000000e+00],
       [7.4102413e-26, 2.3323866e-25, 1.0000000e+00],
       [4.7249593e-25, 7.828

In [57]:
# which ever have highest 
import numpy as np
#y_pred = np.argmax(y_pred, axis=1)
y_pred = np.argmax(y_pred, axis = 0)

In [58]:
y_pred

0

In [28]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [29]:
model=load_model('model_resnet50.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [59]:
model

In [60]:
img_data

array([[[[-102.95077 , -115.790764, -122.691765],
         [-102.95077 , -115.790764, -122.691765],
         [-102.95077 , -115.790764, -122.691765],
         ...,
         [-103.25665 , -116.04175 , -122.91921 ],
         [-103.16645 , -115.95939 , -122.83686 ],
         [-103.06449 , -115.85743 , -122.7349  ]],

        [[-102.95077 , -115.790764, -122.691765],
         [-102.95077 , -115.790764, -122.691765],
         [-102.95077 , -115.790764, -122.691765],
         ...,
         [-103.025276, -115.818214, -122.71137 ],
         [-103.025276, -115.82998 , -122.719215],
         [-103.0292  , -115.8339  , -122.72314 ]],

        [[-102.95077 , -115.790764, -122.691765],
         [-102.95077 , -115.790764, -122.691765],
         [-102.95077 , -115.790764, -122.691765],
         ...,
         [-102.993904, -115.80645 , -122.719215],
         [-102.96645 , -115.79861 , -122.72314 ],
         [-102.96645 , -115.79861 , -122.72314 ]],

        ...,

        [[-103.04096 , -115.97115 , -1

In [61]:
img=image.load_img('Datasets/Test/lamborghini/11.jpg',target_size=(224,224))



In [62]:
x=image.img_to_array(img)
x

array([[[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [194., 188., 174.],
        [215., 209., 197.],
        [241., 235., 223.]],

       [[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [247., 245., 233.],
        [245., 242., 233.],
        [244., 241., 232.]],

       [[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [245., 248., 241.],
        [244., 250., 248.],
        [244., 250., 248.]],

       ...,

       [[190., 206., 229.],
        [190., 206., 229.],
        [190., 206., 229.],
        ...,
        [172., 179., 187.],
        [171., 180., 187.],
        [171., 180., 187.]],

       [[187., 205., 227.],
        [187., 205., 227.],
        [187., 205., 227.],
        ...,
        [172., 179., 187.],
        [171., 180., 187.],
        [171., 180., 187.]],

       [[185., 206., 227.],
        [185., 206., 227.],
        [187., 2

In [63]:
x.shape

(224, 224, 3)

In [64]:
x=x/255

In [65]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [66]:
model.predict(img_data)

array([[8.200653e-15, 6.741602e-13, 1.000000e+00]], dtype=float32)

In [67]:
a=np.argmax(model.predict(img_data), axis=1)

In [68]:
a==1

array([False])